In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions anda
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# Training with pretrained embeddings

## Overview

In this use case we will consider how we might train with pretrained embeddings.

Pretrained embeddings can allow our model to include information from additional modalities (for instance, we might want to grab CNN descriptors of product images). They can also come from other models that we train on our data. For example, we might train a word2vec model on the sequence of purchased items by a customer and want to include this information in our retrieval or ranking model.

The use cases are many, but this particular example will focus on the technical aspects of working with pretrained embeddings.

We will use a synthetic version of the MovieLens 100k dataset and emulate a scenario where we would have a pretrained embedding for each of the movies in the dataset.

### Learning objectives

- Training with pretrained embeddings
- Understanding [the Schema file](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/schema/schema.py)

## Downloading and preparing the dataset

In [2]:
import merlin.models.tf as mm
from merlin.schema.tags import Tags
import tensorflow as tf
from merlin.models.tf.prediction_tasks.classification import BinaryClassificationTask
from merlin.models.tf.blocks import *
from merlin.datasets.synthetic import generate_data

import numpy as np

2022-06-23 04:16:53.178231: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 04:16:53.178616: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 04:16:53.178776: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 04:16:53.200029: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate 

In [3]:
train = generate_data('movielens-100k', num_rows=100_000)

In [4]:
target_column = train.schema.select_by_tag(Tags.TARGET).column_names[1]
target_column

'rating_binary'

The schema holds vital information about our dataset. We can extract the embedding table size for the `moveId` column from it.

In [5]:
train.schema['movieId'].properties['embedding_sizes']['cardinality']

1680.0

From the schema, we can tell that the cardinality of `movieId` is 1680. Index 0 will be used in case an unknown `movieId` is encountered

In order to accommodate this, we initialize our embedding table of dimensionality 1681 by 64.

In [6]:
pretrained_movie_embs = np.random.random((1681, 64))

This is only a mock up embedding table. In reality, this is where we would pass our embeddings from another model.

The dimensionality of each embedding, that of 64, is arbitrary. We could have specified some other value here, though generally multiples of 8 tend to work well.

Let us now feed our embedding table into our model.

## Building the model

We now have everything we need to construct a model and train on our custom embeddings. In order to do so, we will leverage the `TensorInitializer`.

In [7]:
model = mm.DCNModel(
    train.schema,
    depth=2,
    deep_block=mm.MLPBlock([64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(target_column),
    embedding_options=mm.EmbeddingOptions(
        embeddings_initializers={
            "movieId": mm.TensorInitializer(pretrained_movie_embs),
        }
    )
)

2022-06-23 04:16:54.257546: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


We could have created the model without passing anything for the `embedding_options`. The model would still be able to infer how to construct itself (what should be the dimensionality of the input layer and so on) from the information contained in the schema.

Passing a `TensorInitializer` as part of the `embedding_options` tells our model to use our embedding table (`pretrained_movie_embs`) for that particular column of our dataset (`movieId`) as opposed to the model randomly initializing a brand new embedding matrix. For categorical columns where we do not provide this information, the model will go with the standard initialization logic, which is to create an embedding table of appropriate size and perform random preinitialization.

## Training

We train our model with `AUC` as our metric.

As we use synthetic data, the AUC score will not improve significantly.

In [8]:
%%time
opt = tf.keras.optimizers.Adagrad(learning_rate=1e-1)
model.compile(optimizer=opt, run_eagerly=False, metrics=[tf.keras.metrics.AUC()])

CPU times: user 8.02 ms, sys: 24 µs, total: 8.04 ms
Wall time: 7.49 ms


In [9]:
model.fit(train, batch_size=1024, epochs=10)

Epoch 1/10
98/98 [==============================] - 3s 5ms/step - loss: 0.6938 - auc: 0.5015 - regularization_loss: 0.0000e+00
Epoch 2/10
98/98 [==============================] - 0s 5ms/step - loss: 0.6932 - auc: 0.5059 - regularization_loss: 0.0000e+00
Epoch 3/10
98/98 [==============================] - 0s 5ms/step - loss: 0.6930 - auc: 0.5089 - regularization_loss: 0.0000e+00
Epoch 4/10
98/98 [==============================] - 0s 5ms/step - loss: 0.6929 - auc: 0.5123 - regularization_loss: 0.0000e+00
Epoch 5/10
98/98 [==============================] - 0s 5ms/step - loss: 0.6929 - auc: 0.5118 - regularization_loss: 0.0000e+00
Epoch 6/10
98/98 [==============================] - 0s 5ms/step - loss: 0.6928 - auc: 0.5137 - regularization_loss: 0.0000e+00
Epoch 7/10
98/98 [==============================] - 1s 5ms/step - loss: 0.6928 - auc: 0.5149 - regularization_loss: 0.0000e+00
Epoch 8/10
98/98 [==============================] - 0s 5ms/step - loss: 0.6926 - auc: 0.5176 - regularization_l

The model trains and we have utilized pretrained embeddings 